In [1]:
import pandas as pd
import numpy as np
import nltk
import re

In [2]:
import matplotlib.pyplot as plt

In [3]:
# from google.colab import drive
# drive.mount('/content/drive')

In [4]:
df = pd.read_csv("employee_reviews.csv")
#df=pd.read_csv('C:/Users/sangeeth kumar/PycharmProjects/python_class2/employee_reviews.csv')

FileNotFoundError: [Errno 2] File b'employee_reviews.csv' does not exist: b'employee_reviews.csv'

In [ ]:
df.replace('none',np.NaN,inplace=True)

In [ ]:
df.shape

In [ ]:
df.head(3)

In [ ]:
df.isna().sum()

In [ ]:
df.dtypes

In [ ]:
df.columns

In [ ]:
df['summary'].fillna('None',inplace=True)
df['advice-to-mgmt'].fillna('None',inplace=True)
df['cons'].fillna('None',inplace=True)

In [ ]:
df.['dates']

In [ ]:
review_date=[]
for each_date in df['dates']:
    month=(each_date[:4])
    year=(each_date[-4:])
    date=month+' '+year
    review_date.append(date)


In [ ]:
df['review_date']=review_date

In [ ]:
df.drop(['Unnamed: 0','dates','work-balance-stars', 'culture-values-stars',
       'carrer-opportunities-stars', 'comp-benefit-stars',
       'senior-mangemnet-stars','link'],axis=1,inplace=True)

In [ ]:
df['review_date'].nunique()

# EDA

In [ ]:
df.groupby('company')['overall-ratings'].count().plot(kind='bar')

In [ ]:
print(df.groupby(['overall-ratings','company'])['company'].size().unstack(level=1,fill_value=0))
df.groupby(['overall-ratings','company'])['company'].count().unstack().plot(kind='bar',figsize=(10,7))

In [ ]:
current_employees_df=df[df['job-title'].str.startswith('C')]
former_employees_df=df[df['job-title'].str.startswith('F')]

# current employees rating to their companies

In [ ]:
print(current_employees_df.groupby(['overall-ratings','company'])['company'].count().unstack(level=1,fill_value=0))
current_employees_df.groupby(['overall-ratings','company'])['company'].count().unstack().plot(kind='bar',figsize=(10,7))

In [ ]:
df.head(5)

In [ ]:
positive_reviews_df=df.loc[df['overall-ratings']>3.0]
neutral_reviews_df=df.loc[df['overall-ratings']==3.0]
negative_reviews_df=df.loc[df['overall-ratings']<3.0]

In [ ]:
cols=['summary','pros','cons']
positive_df=positive_reviews_df[cols]
neutral_df=neutral_reviews_df[cols]
negative_df=negative_reviews_df[cols]

In [ ]:
positive_df['all_reviews']=positive_df['summary']+" "+positive_df['pros']+" "+positive_df['cons']
positive_df=positive_df['all_reviews']

neutral_df['all_reviews']=neutral_df['summary']+" "+neutral_df['pros']+" "+neutral_df['cons']
neutral_df=neutral_df['all_reviews']

negative_df['all_reviews']=negative_df['summary']+" "+negative_df['pros']+" "+negative_df['cons']
negative_df=negative_df['all_reviews']

In [ ]:
positive_df

In [ ]:
import nltk
nltk.download('all')

from nltk.tokenize import word_tokenize
from nltk.tokenize import sent_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.stem import PorterStemmer


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.tokenize import TweetTokenizer
from wordcloud import WordCloud

In [ ]:
import re
stopwords=nltk.corpus.stopwords.words('english')
lemmatizer=WordNetLemmatizer()

In [ ]:
print(stopwords)

In [ ]:
positive_texts=positive_df.str.cat(sep='.')
neutral_texts=neutral_df.str.cat(sep='.')
negative_texts=negative_df.str.cat(sep='.')

In [ ]:
negative_texts

In [ ]:
def clean_text(text):
    text=re.sub(r"''","",text)
    text=text.replace("Etc.","")
    text=text.replace("etc.","")
    text=re.sub(r"[0-9]",'',text)
    text=re.sub(r'[!?:/)(*+~\"-,]','',text)
    
    # tokenize tweets
    tokenizer = TweetTokenizer(preserve_case=False, strip_handles=True, reduce_len=True)
    tweet_tokens = tokenizer.tokenize(text)
    
    reviews_clean = []    
    for word in tweet_tokens:
        if (word not in stopwords):
            lem_word = lemmatizer.lemmatize(word)
            reviews_clean.append(lem_word)
 
    return reviews_clean

In [ ]:
def bagOfWords(review):
    words=clean_text(review)
    word_dictionary=dict([word,True] for word in words)
    return word_dictionary

In [ ]:
bagOfWords('Night shifts, Work gets a little')

In [ ]:
positive_review_set=[]
for each_review in positive_texts.split('.'):
   positive_review_set.append((bagOfWords(each_review), 'Positive Review'))

In [ ]:
positive_review_set[:2]

In [ ]:
neutral_review_set=[]
for each_review in neutral_texts.split('.'):
   neutral_review_set.append((bagOfWords(each_review), 'Neutral Review'))

In [ ]:
negative_review_set=[]
for each_review in negative_texts.split('.'):
   negative_review_set.append((bagOfWords(each_review), 'Negative Review'))

In [ ]:
from random import shuffle 
shuffle(positive_review_set)
shuffle(neutral_review_set)
shuffle(negative_review_set)
 
train_set = positive_review_set[:150000] + neutral_review_set[:150000] + negative_review_set[:150000]
test_set = positive_review_set[150000:] + neutral_review_set[150000:] + negative_review_set[150000:]
 
print(len(test_set),  len(train_set))

In [ ]:
from nltk import classify
from nltk import NaiveBayesClassifier

classifier = NaiveBayesClassifier.train(train_set)
 
accuracy = classify.accuracy(classifier, test_set)
print(accuracy*100)

In [ ]:
custom_review="a less work load and easy to get everything completed"

cr=bagOfWords(custom_review)
print(cr)
print(classifier.classify(cr))